## 实验题目5 高斯(Gauss)列主元消去法

### 代码实现

In [33]:
using Printf
using LinearAlgebra

In [34]:
# from: https://stackoverflow.com/questions/58667332/is-there-a-way-to-swap-columns-in-o1-in-julia
function swapcols!(X::AbstractMatrix, i::Integer, j::Integer)
    @inbounds for k = 1:size(X, 1)
        X[k, i], X[k, j] = X[k, j], X[k, i]
    end
end
# from: https://discourse.julialang.org/t/swap-cols-rows-of-a-matrix/47904/9
function _swapcol!(x, i, j)
    for k in axes(x, 1)  # <- give dimension as input to axes function
        x[k, i], x[k, j] = x[k, j], x[k, i]
    end
end

_swapcol! (generic function with 1 method)

In [35]:
function swaprows!(X::AbstractMatrix, i::Integer, j::Integer)
    @inbounds for k = 1:size(X, 2)
        X[i, k], X[j, k] = X[j, k], X[i, k]
    end
end

swaprows! (generic function with 1 method)

In [36]:
# https://stackoverflow.com/questions/45396685/what-does-an-exclamation-mark-mean-after-the-name-of-a-function
# https://people.richland.edu/james/lecture/m116/matrices/pivot.html
function pivoting!(A::Matrix{Float64}, k::Integer, n::Integer)
    val, idx = findmax(A[k:n, k])
    idx += k - 1  # index must add previous length that omitted by slice operator
    return val, idx
end
function pivoting!(A::Matrix{Float64}, b::Vector{Float64}, k::Integer, n::Integer, implicit::Bool)
    s = [maximum(A[i, k:n]) for i in k:n]
    if 0 in s
        println("Cannot solve a singular matrix!")
        return
    end
    if implicit
        val, idx = findmax(A[k:n, k] ./ s[1:n-k+1])
    else
        A[k:n, k:n] = A[k:n, k:n] ./ s
        b[k:n] = b[k:n] ./ s
        val, idx = findmax(A[k:n, k])
    end
    idx += k - 1  # index must add previous length that omitted by slice operator
    return val, idx
end

pivoting! (generic function with 2 methods)

In [37]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss(n, A::Matrix{Float64}, b::Vector{Float64})
    for k = 1:n-1
        # select pivot in columns
        val, idx = pivoting!(A, k, n)
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss (generic function with 2 methods)

In [38]:
# Gauss列主元消去法
# Todo: modify it using . operator
function gauss(n, A::Matrix{Float64}, b::Vector{Float64}, implicit::Bool)
    for k = 1:n-1
        # select pivot in columns
        val, idx = pivoting!(A, b, k, n, implicit)
        if val == 0
            println("Cannot solve a singular matrix!")
            return
        end
        # swap rows
        if idx != k
            swaprows!(A, idx, k)
            b[idx], b[k] = b[k], b[idx]
        end
        # elimination
        for i = k+1:n
            m = A[i, k] / A[k, k]
            A[i, :] -= A[k, :] * m
            b[i] -= b[k] * m
        end
    end
    if A[n, n] == 0
        println("Cannot solve a singular matrix!")
        return
    end
    # https://stackoverflow.com/questions/62142717/julia-quick-way-to-initialise-an-empty-array-thats-the-same-size-as-another
    x = similar(b, Float64)
    x[n] = b[n] / A[n, n]
    for k = n-1:-1:1  # the usage of reverse sequence
        x[k] = (b[k] - dot(A[k, k+1:n], x[k+1:n])) / A[k, k]  # something really annoying 
    end
    x
end


gauss (generic function with 2 methods)

### 测试代码

#### Test 1 - Correctness

In [43]:
# test random result of standard library 
# test pass
for i in 1:5
    M = rand(300, 300)
    v = rand(300)
    A, b = copy(M), copy(v)  # ? Todo: move this line into try block will extend compilation time
    try
        @time print("$(norm(A \ b - gauss(size(A, 1), A, b), 2))\t")

        A, b = copy(M), copy(v)
        @time print("$(norm(A \ b - gauss(size(A, 1), A, b, false), 2))\t")  # implicit=false

        A, b = copy(M), copy(v)
        @time print("$(norm(A \ b - gauss(size(A, 1), A, b, true), 2))\t")  # implicit=true
    catch SigularException
        println("Cannot solve a singular matrix!")
    end
    println()
end

1.5784558737083342e-12	  0.165447 seconds (180.32 k allocations: 439.810 MiB, 18.27% gc time)
2.2105775350560245e-12	  0.238583 seconds (228.37 k allocations: 653.650 MiB, 14.48% gc time)
3.747745877978703e-12	  0.205463 seconds (227.26 k allocations: 515.581 MiB, 16.05% gc time)

4.2734920856226555e-13	  0.159513 seconds (180.32 k allocations: 439.810 MiB, 18.20% gc time)
8.453040626029741e-13	  0.243773 seconds (228.37 k allocations: 653.650 MiB, 14.33% gc time)
1.028586353582166e-12	  0.210639 seconds (227.26 k allocations: 515.581 MiB, 16.21% gc time)

5.360199735445008e-12	  0.200887 seconds (180.32 k allocations: 439.810 MiB, 18.30% gc time)
3.319134663847233e-11	  0.252332 seconds (228.37 k allocations: 653.650 MiB, 13.92% gc time)
4.7140376955827686e-11	  0.205224 seconds (227.26 k allocations: 515.581 MiB, 15.11% gc time)

4.077611796649429e-13	  0.177729 seconds (180.32 k allocations: 439.810 MiB, 17.04% gc time)
2.4156478552054574e-12	  0.245946 seconds (228.37 k allocations

#### Test 2 - Performance 

In [40]:
# test random result of standard library 
# test pass
for i in 1:5
    M = rand(300, 300)
    v = rand(300)
    A, b = copy(M), copy(v)
    try
        @time A \ b

        A, b = copy(M), copy(v)
        @time gauss(size(A, 1), A, b)

        A, b = copy(M), copy(v)
        @time gauss(size(A, 1), A, b, false)  # implicit=false

        A, b = copy(M), copy(v)
        @time gauss(size(A, 1), A, b, true)  # implicit=true
    catch SigularException
        println("Cannot solve a singular matrix!")
    end
    println()
end

  0.007170 seconds (4 allocations: 708.172 KiB)
  0.335789 seconds (180.30 k allocations: 439.115 MiB, 56.10% gc time)
  0.243270 seconds (228.35 k allocations: 652.955 MiB, 14.83% gc time)
  0.209229 seconds (227.24 k allocations: 514.886 MiB, 15.83% gc time)

  0.006420 seconds (4 allocations: 708.172 KiB)
  0.155101 seconds (180.30 k allocations: 439.115 MiB, 19.02% gc time)
  0.231446 seconds (228.35 k allocations: 652.955 MiB, 15.36% gc time)
  0.199493 seconds (227.24 k allocations: 514.886 MiB, 15.89% gc time)

  0.005846 seconds (4 allocations: 708.172 KiB)
  0.154079 seconds (180.30 k allocations: 439.115 MiB, 18.81% gc time)
  0.234519 seconds (228.35 k allocations: 652.955 MiB, 14.52% gc time)
  0.204228 seconds (227.24 k allocations: 514.886 MiB, 16.35% gc time)

  0.006043 seconds (4 allocations: 708.172 KiB)
  0.153117 seconds (180.30 k allocations: 439.115 MiB, 18.73% gc time)
  0.231499 seconds (228.35 k allocations: 652.955 MiB, 14.71% gc time)
  0.193914 seconds (227.

#### Test 3 - Special Matrix

Bug: 处理上三角矩阵计算有异常

Todo: 应当增加对于特殊矩阵的测试，待测试列表同Julia Special Matrix:

- [ ] Symmetric
- [ ] Hermitian
- [ ] UpperTriangular
- [ ] UnitUpperTriangular
- [ ] LowerTriangular
- [ ] UnitLowerTriangular
- [ ] UpperHessenberg
- [ ] Tridiagonal
- [ ] SymTridiagonal
- [ ] Bidiagonal
- [ ] Diagonal
- [ ] UniformScaling

基本语法为 `M = SpecialMatrix(rand(300,300))`

### 实验题目

#### 问题 1

In [50]:
A = [0.4096 0.1234 0.3678 0.2943
    0.2246 0.3872 0.4015 0.1129
    0.3645 0.1920 0.3781 0.0643
    0.1784 0.4002 0.2786 0.3927]
b = [1.1951; 1.1262; 0.9989; 1.2499]
display(@time A \ b)
display(@time gauss(4, A, b))
# display(@time gauss(4, A, b, false))  # implicit=false
# display(@time gauss(4, A, b, true))  # implicit=true

4-element Vector{Float64}:
 1.0000000000000027
 1.0000000000000018
 0.9999999999999971
 0.9999999999999992

4-element Vector{Float64}:
 1.000000000000001
 1.000000000000001
 0.9999999999999987
 0.9999999999999996

4-element Vector{Float64}:
 1.000000000000001
 1.000000000000001
 0.9999999999999987
 0.9999999999999996

  0.000022 seconds (3 allocations: 384 bytes)
  0.000011 seconds (34 allocations: 3.031 KiB)
  0.000023 seconds (67 allocations: 5.594 KiB)
  0.000017 seconds (61 allocations: 4.781 KiB)


In [51]:
A = [136.01 90.860 0 0
    90.860 98.810 -67.590 0
    0 -67.590 132.01 46.260
    0 0 46.260 177.17]
b = [226.87; 122.08; 110.68; 223.43]
display(@time A \ b)
display(@time gauss(4, A, b))
# display(@time gauss(4, A, b, false))  # implicit=false
# display(@time gauss(4, A, b, true))  # implicit=true

4-element Vector{Float64}:
 1.0000000000000704
 0.9999999999998946
 0.9999999999999408
 1.0000000000000155

4-element Vector{Float64}:
 1.000000000000133
 0.9999999999998009
 0.999999999999888
 1.0000000000000293

  0.000021 seconds (3 allocations: 384 bytes)
  0.000013 seconds (34 allocations: 3.031 KiB)


In [53]:
A = [1 1/2 1/3 1/4
     1/2 1/3 1/4 1/5
     1/3 1/4 1/5 1/6
     1/4 1/5 1/6 1/7]
b = [25 / 12; 77 / 60; 57 / 60; 319 / 420]
display(@time A \ b)
display(@time gauss(4, A, b))
# display(@time gauss(4, A, b, false))  # implicit=false
# display(@time gauss(4, A, b, true))  # implicit=true

4-element Vector{Float64}:
 0.9999999999999779
 1.000000000000241
 0.9999999999994366
 1.0000000000003588

4-element Vector{Float64}:
 0.9999999999999927
 1.0000000000000688
 0.9999999999998556
 1.0000000000000846

  0.000032 seconds (3 allocations: 384 bytes)
  0.000012 seconds (34 allocations: 3.031 KiB)


In [55]:
A = [10 7 8 7
    7 5 6 5
    8 6 10 9
    7 5 9 10]
b = [32; 23; 33; 31]
display(@time A \ b)
display(@time gauss(4, A, b))
# display(@time gauss(4, A, b, false))  # implicit=false
# display(@time gauss(4, A, b, true))  # implicit=true

4-element Vector{Float64}:
 1.000000000000083
 0.9999999999998619
 1.000000000000035
 0.999999999999979

  0.000020 seconds (3 allocations: 384 bytes)


LoadError: MethodError: no method matching gauss(::Int64, ::Matrix{Int64}, ::Vector{Int64})
[0mClosest candidates are:
[0m  gauss(::Any, [91m::Matrix{Float64}[39m, [91m::Vector{Float64}[39m) at In[37]:3
[0m  gauss(::Any, [91m::Matrix{Float64}[39m, [91m::Vector{Float64}[39m, [91m::Bool[39m) at In[38]:3